In [1]:
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Imports

In [2]:
# External libraries
import os as so
import sys as s
import pathlib as pl
import torch
import torch.nn as nn
from torch import Tensor
from torch.utils.data import random_split
from torch.utils.data import DataLoader, ConcatDataset
import torcheval
from torcheval.metrics import MulticlassF1Score, Mean
import optuna as opt
import torchvision as tn
import sklearn as sn
from sklearn.metrics import f1_score
import pandas as ps
import numpy as ny
import typing as t
import pathlib as pl
import matplotlib.pyplot as pt
import random as rng
from tqdm import tqdm
import tqdm as tm
from pprint import pprint
from git import Repo

In [3]:
# Add local package to path
if (p := pl.Path(so.getcwd(), '..').absolute().as_posix()) not in s.path:
    s.path.append(p)

# Local imports
from gic import *
from gic.data import GenImageDataset
from gic.tune import ClassificationTrainer
from gic.models.resnet import ResCNN, resnet_sampler
from gic.models.densenet import DenseCNN, densenet_sampler

wandb: Currently logged in as: invokariman (castelvaar). Use `wandb login --relogin` to force relogin
/home/invokariman/Projects/git/ub-g21-deeplearning/projects/gic/notebooks/../gic/__init__.py:57: ExperimentalWarning: WeightsAndBiasesCallback is experimental (supported from v2.9.0). The interface can change in the future.
  wn_callback = WeightsAndBiasesCallback(
/home/invokariman/Projects/git/ub-g21-deeplearning/projects/gic/notebooks/../gic/tune.py:114: ExperimentalWarning: track_in_wandb is experimental (supported from v3.0.0). The interface can change in the future.
  @forward_self(wn_callback.track_in_wandb())


HyperParameter Optimization

In [4]:
# Find the best model using a trainer
trainer = ClassificationTrainer(
    seed=SEED,
    device=DEVICE,
    model=DenseCNN,
    profiling=profiling,
    mix_float=mix_float,
    hps=densenet_sampler,
    dataset_path=DATA_PATH,
    num_workers=num_workers,
    prefetch_factor=prefetch_factor,
)

In [5]:
# Search hyperparams
searcher = opt.samplers.TPESampler(n_startup_trials=5)
search = opt.create_study(direction='maximize', storage=db_uri, sampler=searcher)
search.optimize(trainer, n_trials=30, callbacks=[wn_callback])
wn.finish()

[I 2023-12-01 23:24:52,628] A new study created in RDB with name: no-name-e33df37f-feab-45ba-add0-4f1f76b04a3f


DenseCNN(
  (input_conv): Conv2d(3, 48, kernel_size=(1, 1), stride=(1, 1))
  (dense_layers): Sequential(
    (0): DenseBlock(
      (layers): ModuleList(
        (0): DenseCompConvBlock(
          (layers): Sequential(
            (comp_block): DenseConvBlock(
              (bn_layer): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (activ_fn): GELU(approximate='none')
              (conv_layer): Conv2d(48, 36, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (drop): Dropout2d(p=0.249587398118041, inplace=False)
            )
            (conv_block): DenseConvBlock(
              (bn_layer): BatchNorm2d(36, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (activ_fn): GELU(approximate='none')
              (conv_layer): Conv2d(36, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (drop): Dropout2d(p=0.249587398118041, inplace=False)
            )
          )
        )
  

epochs:   0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
search.best_params

Final Training

In [ ]:
# Use best model parameters
params = {
    'batch_size': 16,
    'optimizer': 'AdamW',
    'lr': 3e-4,
    'epochs': 50,
    'weight_decay': 5e-5,
    'pool': 'avg',
    'dense_dropout': 0.3,
    'conv_dropout': 0.4,
    'conv_chan': 32,
    'dens_chan': 512,
    'activ': 'SiLU'
}

# Train on `almost` the whole labeled dataset
trainer.train(params)
wn.finish()

Perform evaluation

In [ ]:
preds: t.List[int] = trainer.eval(params).tolist()
preds[:10]

In [ ]:
len(preds)

Save Submission

In [ ]:
test_data = GenImageDataset(DATA_PATH, 'test', True)
dataf = {'Image': [], 'Class': []}
dataf['Class'] = preds
test_data._GenImageDataset__data['Class'] = preds
test_data._GenImageDataset__data.to_csv(SUBMISSION_PATH, index=False)